Homework #5 - Numerical Methods - Mariana Baca - 9/25/25-

3.3: Problem 8, Problem 18

3.4: Problem 6


Problem 3.3-8. a) Use Algorithm 3.2 to construct the interpolating polynomial of degree four for the unequally spaced points given in the following table:

\\begin{table}[]
\begin{tabular}{ll}
x   & f(x)     \\
0.0 & -6.00000 \\
0.1 & -5.89483 \\
0.3 & -5.65014 \\
0.6 & -5.17788 \\
1.0 & -4.28172
\end{tabular}
\end{table}

b) Add f(1.1) = -3.99583 to the table and construct the interpolating polynomial of degree five.



In [26]:
#Problem 3.3-8.
import numpy as np
import matplotlib.pyplot as plt
import math

def calculate_divided_differences(x_nodes, f_x_nodes):

    n = len(x_nodes)
    divided_diffs = np.zeros((n, n))
    divided_diffs[:, 0] = f_x_nodes  # Initialize the first column with f(x) values

    for j in range(1, n):
        for i in range(n - j):
            divided_diffs[i, j] = (divided_diffs[i + 1, j - 1] - divided_diffs[i, j - 1]) / (x_nodes[i + j] - x_nodes[i])

    return divided_diffs

def newton_interpolating_polynomial_expression(x_nodes, divided_diffs, tolerance=1e-9):

    coefficients = np.diag(divided_diffs)
    n = len(x_nodes)
    poly_parts = []

    if abs(coefficients[0]) > tolerance:
        poly_parts.append(f"{coefficients[0]:.6f}") ##Only adds terms if coefficent isn't zero.

    for j in range(1, n):
        if abs(coefficients[j]) > tolerance:
            term = f"{abs(coefficients[j]):.6f}" # Use absolute value for term coefficient
            for i in range(j):
                if x_nodes[i] >= 0:
                    term += f"*(x - {x_nodes[i]:.1f})"
                else:
                    term += f"*(x + {-x_nodes[i]:.1f})"

            if coefficients[j] >= 0: # adds a minus or a plus for negative/positive coefficients
                poly_parts.append(f"+ {term}")
            else:
                poly_parts.append(f"- {term}")

    # Join the parts to form the polynomial
    if not poly_parts:
        return "0"
    else:
        return " ".join(poly_parts).lstrip('+ ')

### Data for part a)
x_a = np.array([0.0, 0.1, 0.3, 0.6, 1.0])
fx_a = np.array([-6.00000, -5.89483, -5.65014, -5.17788, -4.28172])

### calculate divided differences for part a)
F_a = calculate_divided_differences(x_a, fx_a)
diag_a = np.diag(F_a)

print("Divided Difference Table for (a):")
print(F_a)
print("\nDiagonal coefficients for (a): [F_(0,0), F_(1,1), F_(2,2), F_(3,3), F_(4,4)]")
print(diag_a)

# Construct and print the interpolating polynomial expression for part (a)
P4_expression = newton_interpolating_polynomial_expression(x_a, F_a)
print("\nInterpolating Polynomial for (a):")
print("Raw P4(x) =", P4_expression)
print("")
print("Simplified: P4(x) = 0.951714x^2 - 1.1420868x - 6")

### Part (b)

xb = np.append(x_a, 1.1)
fxb = np.append(fx_a, -3.99583)

# Calculate divided differences for part b)
Fb = calculate_divided_differences(xb, fxb)
diagb = np.diag(Fb)

print("\nDivided Difference Table for (b):")
print(Fb)
print("\nDiagonal coefficients for (b):")
print(diagb)

# Construct and print the interpolating polynomial expression for part (b)
P5_expression = newton_interpolating_polynomial_expression(xb, Fb)
print("\nInterpolating Polynomial for (b):")
print(f"Raw expression for P5(x) = {P5_expression}")
print("")
print("Simplified: P5(x) = 0.951714x^2 + 1.1282786 - 6 ")

Divided Difference Table for (a):
[[-6.          1.0517      0.5725      0.215       0.06301587]
 [-5.89483     1.22345     0.7015      0.27801587  0.        ]
 [-5.65014     1.5742      0.95171429  0.          0.        ]
 [-5.17788     2.2404      0.          0.          0.        ]
 [-4.28172     0.          0.          0.          0.        ]]

Diagonal coefficients for (a): [F_(0,0), F_(1,1), F_(2,2), F_(3,3), F_(4,4)]
[-6.          1.22345     0.95171429  0.          0.        ]

Interpolating Polynomial for (a):
Raw P4(x) = -6.000000 + 1.223450*(x - 0.0) + 0.951714*(x - 0.0)*(x - 0.1)

Simplified: P4(x) = 0.951714x^2 - 1.1420868x - 6

Divided Difference Table for (b):
[[-6.          1.0517      0.5725      0.215       0.06301587  0.01415945]
 [-5.89483     1.22345     0.7015      0.27801587  0.07859127  0.        ]
 [-5.65014     1.5742      0.95171429  0.35660714  0.          0.        ]
 [-5.17788     2.2404      1.237       0.          0.          0.        ]
 [-4.28172     2

Problem 3.3-18. The fastest time ever recorded in the Kentucky Derby was by a horse names Secretariat in 1973. He covered the 1 1/4 mile track in 1:59 2/5 (1 minute, 59.4 seconds). Times at the quarter-mile, halge mile, and mile poles were 0:25 1/5, 0:49 1/5, and 1:36 2/5.

a) Use interpolation to predict the time at the three-quarter mile pole and compare it to the actual time of 1:13.

b) Use the derivative of the interpolating polynomial to estimate the speed of Secretariate at the end of the race.



In [27]:
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy.polynomial import Polynomial

def divided_differences(x, y):

    x = np.asarray(x)
    coef = np.array(y).copy()
    n = len(x)
    for j in range(1, n):
        coef[j:n] = (coef[j:n] - coef[j-1:n-1]) / (x[j:n] - x[:n-j])
    return coef

def newton_Polynomial(coeffs, x_nodes):
  ###  Constructs the Newton interpolating polynomial using np.polynomial.Polynomial.
    p = Polynomial([coeffs[0]])
    basis = Polynomial([1.0])

    for k in range(1, len(coeffs)):
        basis = basis * Polynomial([-x_nodes[k-1], 1.0])
        p = p + coeffs[k] * basis

    return p

# Distances (miles)
x = np.array([0.00, 0.25, 0.50, 1.00, 1.25])

# Times (seconds): 0:25 1/5 = 25.2, 0:49 1/5 = 49.2, 1:36 2/5 = 96.4, 1:59 2/5 = 119.4
t = np.array([0.0, 25.2, 49.2, 96.4, 119.4])

##Polynomial P
dd = divided_differences(x, t)
P = newton_Polynomial(dd, x)

# Get the derivative using the .deriv() function from np.Polynomial
dP = P.deriv()

#### Part (a) time at 0.75 miles
xq = 0.75
pred_time = float(P(xq))
actual_time = 73.0               # 1:13 = 73 s
diff = pred_time - actual_time

print(f"(a) Predicted time at 0.75 mi: {pred_time} s")
print(f"    Actual time at 0.75 mi:    {actual_time} s")
print(f"    Difference (pred-actual):  {diff} s")

### Part (b) speed at finish (x = 1.25 mi)
dt_dx_finish = float(dP(1.25))   # seconds per mile
speed_mps = 1.0 / dt_dx_finish   # miles per second
speed_mph = speed_mps * 3600.0   # miles per hour

print(f"(b) Estimated speed at finish: {speed_mph:} mph")


(a) Predicted time at 0.75 mi: 72.86 s
    Actual time at 0.75 mi:    73.0 s
    Difference (pred-actual):  -0.14000000000000057 s
(b) Estimated speed at finish: 40.12483281319661 mph


3.4.6- Let f(x) = 3xe^x - e^(2x).
a) Approximate f(1.03) by the Hermite interpolating polynomial of degree at most three using x0 = 1 and x1 = 1.05. Compute the actual error to the error bound.

b) Repeat (a) with the Hermite interpolating polynomial of degree at most five using x0 = 1, x1 = 1.05, and x2 = 1.07.

In [22]:
### Problem 3.4.6-
import math
import numpy as np
import matplotlib.pyplot as plt

def HermiteInterpolation(x, x_nodes, f, df):

    n = len(x_nodes) - 1
    m = 2 * (n + 1)
    z = np.zeros(m)
    a = np.zeros(m)
    Q = np.zeros((m, m))

    for i in range(n + 1):
        z[2*i] = x_nodes[i]
        z[2*i + 1] = x_nodes[i]
        Q[2*i, 0] = f(x_nodes[i])
        Q[2*i + 1, 0] = f(x_nodes[i])
        a[2*i] = f(x_nodes[i])

    for i in range(n):
        Q[2*i + 1, 1] = df(x_nodes[i])

    for i in range(1, 2 * n + 1):
        for j in range(1, i + 1):
            if Q[i, j-1] == Q[i-1, j-1] and z[i] == z[i-j]: ##If all x values are distinct then this is unnecessary
                pass
            elif z[i] == z[i-j]:
                 ## handles when the same x node is repeated (for derivative)
                 Q[i, j] = df(z[i]) if j == 1 else (Q[i, j-1] - Q[i-1, j-1]) / (z[i] - z[i-j])
            else:
                 Q[i, j] = (Q[i, j-1] - Q[i-1, j-1]) / (z[i] - z[i-j])

    ### extract the coefficients from the diagonal of Q
    for i in range(m):
        a[i] = Q[i, i]

    # Evaluate
    hermite_poly = a[0]
    product_term = 1.0
    for i in range(1, m):
        product_term *= (x - z[i-1])
        hermite_poly += a[i] * product_term

    return hermite_poly

# Define f and f'
def f(x):
    return 3 * x * math.exp(x) - math.exp(2 * x)

def df(x):
    return 3 * math.exp(x) + 3 * x * math.exp(x) - 2 * math.exp(2 * x)


# Data for part (a)
x_nodes_a = np.array([1.0, 1.05])

x_evaluate_a = 1.03
approx_a = HermiteInterpolation(x_evaluate_a, x_nodes_a, f, df)

print(f"(a) Approximation of f(1.03) using x0=1 and x1=1.05: {approx_a:.6f}")

# Compute the actual value and error
actual_value_a = f(x_evaluate_a)
actual_error_a = abs(actual_value_a - approx_a)
print(f" Actual value of f(1.03): {actual_value_a:.6f}")
print(f" Actual error: {actual_error_a:.6f}")


### Part b)

# Data for part (b)
x_nodes_b = np.array([1.0, 1.05, 1.07])

x_evaluate_b = 1.03
approx_b = HermiteInterpolation(x_evaluate_b, x_nodes_b, f, df)

print(f"(b) Approximation of f(1.03) using x0=1, x1=1.05, and x2=1.07: {approx_b}")

# Compute the actual value and error for part (b)
actual_value_b = f(x_evaluate_b)
actual_error_b = abs(actual_value_b - approx_b)
print(f" Actual value of f(1.03): {actual_value_b}")
print(f"Actual error: {actual_error_b}")



(a) Approximation of f(1.03) using x0=1 and x1=1.05: 0.809239
 Actual value of f(1.03): 0.809324
 Actual error: 0.000084
(b) Approximation of f(1.03) using x0=1, x1=1.05, and x2=1.07: 0.8093235946703151
 Actual value of f(1.03): 0.8093236189017059
Actual error: 2.4231390782780693e-08
